In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
plt.style.available


['seaborn-paper',
 'fivethirtyeight',
 'seaborn-poster',
 'bmh',
 'seaborn-dark-palette',
 'seaborn-bright',
 'seaborn-deep',
 'ggplot',
 'classic',
 'seaborn',
 'seaborn-dark',
 'seaborn-ticks',
 'seaborn-pastel',
 'seaborn-muted',
 'seaborn-darkgrid',
 'tableau-colorblind10',
 'Solarize_Light2',
 'seaborn-white',
 'seaborn-talk',
 'seaborn-notebook',
 'seaborn-colorblind',
 'dark_background',
 'grayscale',
 'fast',
 '_classic_test',
 'seaborn-whitegrid']

In [84]:
import os
csv = dict()

def register(project_name):
    csv_name = project_name + '_output.txt'
    csv_path = os.path.join(os.path.expanduser('~'), 'Dropbox','shared','Python scripts', 'LazyUT_TEST', csv_name)
    csv[project_name] = pd.read_csv(csv_path)

projects = ['Abseil']
for p in projects:
    register(p)

In [86]:
class ModeData:
    def __init__(self, mode):
        self.mode = mode
        self.v_num_test = []
        self.v_preprocessor_time = []
        self.v_lazyut_time = []
        self.v_testing_time = []
        self.v_cum_testing_time = []
    def size(self):
        return len(self.v_testing_time)
    def name(self):
        if self.mode == 'test-affected':
            return 'LazyUT'
        if self.mode == 'test-all':
            return 'RetestAll'
        return 'unknown'

def make_mode_data(csv, mode):
    mode_data = ModeData(mode)
    csv_commits = csv.loc[csv['MODE'] == mode].groupby('COMMIT')
    cum_time = 0
    for commit, data in csv_commits:
        test_time = data.TEST.mean()
        if mode == 'test-affected':
            test_time += data.ANALYSIS.mean()
        cum_time += test_time
        mode_data.v_testing_time.append(test_time)
        mode_data.v_cum_testing_time.append(cum_time)
        mode_data.v_num_test.append(data.FILES.max())
        mode_data.v_preprocessor_time.append(data.PREPROCESSOR.mean())
        mode_data.v_lazyut_time.append(data.ANALYSIS.mean())
    return mode_data

def listToLatexCoord(lst):
    result_str = ''
    for i, val in enumerate(lst):
        result_str += '({}, {}) '.format(i + 1, val)
    return result_str
def getVar(var_name, data, project_name, modeName = "", method = listToLatexCoord):
    return {'var{}{}{}'.format(project_name, modeName, var_name) :
            '{' + str(method(data)) + '}'}
def getTicks(size, project_name):
    def listWithoutBrackets(lst):
        result_str = ''
        for val in lst:
            if result_str:
                result_str += ', '
            result_str += str(val)
        return result_str
    return getVar('Ticks', range(1, size + 1), project_name, method=listWithoutBrackets)

def makeLatexVariables(v_mode_data, project_name):
    variables = dict()
    variables.update(getTicks(v_mode_data[0].size(), project_name))
    max_accum = max(v_mode_data[1].v_cum_testing_time)
    max_lazyut_accum = max(v_mode_data[0].v_cum_testing_time)
    variables['efficiency'] = round(max_lazyut_accum / max_accum, 2)
    for mode_data in v_mode_data:
        variables.update(getVar('Test', mode_data.v_testing_time, project_name, mode_data.name()))
        variables.update(getVar('TestAccum', [x / max_accum for x in mode_data.v_cum_testing_time], project_name, mode_data.name()))
        if mode_data.name() == 'LazyUT':
            variables.update(getVar('Analysis', mode_data.v_lazyut_time, project_name, mode_data.name()))
        else:
            variables.update(getVar('Preprocessor', mode_data.v_preprocessor_time, project_name, mode_data.name()))
        variables.update(getVar('NumTest', mode_data.v_num_test, project_name, mode_data.name()))
    return variables

def printLatex(v_mode_data, project_name):
    variables = makeLatexVariables(v_mode_data, '')
    variables.update(dict(lazyut_color = 'Turquoise',
    retestall_color = 'Salmon',
    analysis_color = 'Gray',
    preprocessor_color = 'Fuchsia',
    project_name=project_name))
    
    variables['xmax'] = v_mode_data[0].size() + 0.5
    
    text_form = '''
\\begin{{tikzpicture}}
\\begin{{groupplot}}[
group style={{
    group name=my plots,
    group size=1 by 3,
    xlabels at=edge bottom,
    xticklabels at=edge bottom,
    ylabels at=edge left,
    yticklabels at=edge left,
    vertical sep=1.5cm}},
  footnotesize,
  width=\\textwidth,
  height=5cm,
  xlabel=Порядковый номер версии,
  xmin=0.5,
  xmax={xmax},
  xtick/.expanded={varTicks},
  tickpos=left,
  ytick align=outside,
  xtick align=outside]
\\nextgroupplot[title={{Время тестирования}}]
\\addplot [very thick, {lazyut_color}] coordinates{varLazyUTTest};\label{{plots:testLazy{project_name}}}
\\addplot [very thick, {retestall_color}] coordinates{varRetestAllTest};\label{{plots:testAll{project_name}}}
\\addplot [very thick, {analysis_color}] coordinates{varLazyUTAnalysis};\label{{plots:analysis{project_name}}}
\\addplot [very thick, {preprocessor_color}] coordinates{varRetestAllPreprocessor};\label{{plots:preprocessor{project_name}}}
\\coordinate (top) at (rel axis cs:0,1);% coordinate at top of the first plot
\\nextgroupplot[title={{Число тестовых файлов}}]
\\addplot [very thick, {lazyut_color}]coordinates{varLazyUTNumTest};
\\addplot [very thick, {retestall_color}]coordinates{varRetestAllNumTest};
\\nextgroupplot[title={{Совокупное время тестирования}}]
\\addplot [very thick, {lazyut_color}]coordinates{varLazyUTTestAccum};
\\addplot [very thick, {retestall_color}]coordinates{varRetestAllTestAccum};
\\coordinate (bot) at (rel axis cs:1,0);% coordinate at bottom of the last plot
\\coordinate (A) at (axis cs: 0 , {efficiency});
\\coordinate (O1) at (rel axis cs:0,0);
\\coordinate (O2) at (rel axis cs:1,0);
\\draw [RoyalBlue,dashed] (A -| O1) node [xshift=1.5cm, above] {{ускорение {efficiency} \\%}} -- (A -| O2);
\\end{{groupplot}}
% legend
        \\path (top|-current bounding box.north)--
                    coordinate(legendpos)
                    (bot|-current bounding box.north);
        \\matrix[
                matrix of nodes,
                anchor=south,
                draw,
                inner sep=0.2em,
                draw
            ]at([yshift=1ex]legendpos)
            {{
                \\ref{{plots:testLazy{project_name}}}& LazyUT&[5pt]
                \\ref{{plots:analysis{project_name}}}& Время анализа&[5pt] \\\\
                \\ref{{plots:testAll{project_name}}}& Полное тестирование&[5pt] 
                \\ref{{plots:preprocessor{project_name}}}& Время препроцессинга \\\\}};
\\end{{tikzpicture}}'''
    print(text_form.format(**variables))
    
def make_plots(project):
    csv_proj = csv[project] 
    def modes_data(csv, modes):
        for mode_str in modes:
            yield make_mode_data(csv, mode_str)
    
    plt.figure(figsize=(17,8))
    
    affected_mode_data = make_mode_data(csv_proj, 'test-affected')
    all_mode_data = make_mode_data(csv_proj, 'test-all')
    v_mode_data = [affected_mode_data, all_mode_data]
    
    printLatex(v_mode_data, project)
    return
    
    ax1 = plt.subplot(3, 1, 1)
    ax1.set_title('Время тестирования')
    for mode_data in v_mode_data:
        plt.plot(range(0, mode_data.size()), mode_data.v_testing_time, label=mode_data.name())
        plt.setp(ax1.get_xticklabels(), visible=False)
    plt.plot(all_mode_data.v_preprocessor_time, label='Время препроцессинга')
    plt.plot(affected_mode_data.v_lazyut_time, label='Время анализа')
    plt.legend(loc='upper center', bbox_to_anchor=(0.85, 1.55), prop={'size': 16},
        ncol=2, fancybox=True, shadow=True)
    
    ax2 = plt.subplot(3, 1, 2, sharex = ax1)
    ax2.set_title('Совокупное время тестирования')
    for mode_data in v_mode_data:
        plt.plot(mode_data.v_cum_testing_time, label=mode_data.mode)
        plt.setp(ax2.get_xticklabels(), visible=False)
        
    ax3 = plt.subplot(3, 1, 3, sharex = ax1)
    ax3.set_title('Число тестовых файлов')
    for mode_data in v_mode_data:
        plt.plot(mode_data.v_num_test, label=mode_data.mode)
        plt.setp(ax3.get_xticklabels(), fontsize=10)
        
    plt.show()
    # number of test files
    # testing time
    # cumulative testing time

for p in projects:
    print('---- PROJECT {} ----'.format(p))
    make_plots(p)

---- PROJECT Abseil ----

\begin{tikzpicture}
\begin{groupplot}[
group style={
    group name=my plots,
    group size=1 by 3,
    xlabels at=edge bottom,
    xticklabels at=edge bottom,
    ylabels at=edge left,
    yticklabels at=edge left,
    vertical sep=1.5cm},
  footnotesize,
  width=\textwidth,
  height=5cm,
  xlabel=Порядковый номер версии,
  xmin=0.5,
  xmax=36.5,
  xtick/.expanded={1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36},
  tickpos=left,
  ytick align=outside,
  xtick align=outside]
\nextgroupplot[title={Время тестирования}]
\addplot [very thick, Turquoise] coordinates{(1, 251.57285000000002) (2, 0.9831715) (3, 11.8873) (4, 370.6188) (5, 250.60795000000002) (6, 362.17575) (7, 384.92280000000005) (8, 390.28975) (9, 373.855625) (10, 0.82063) (11, 390.05285000000003) (12, 393.76520000000005) (13, 361.1435) (14, 361.79001) (15, 373.89455) (16, 369.81771) (17, 378.35785) (18, 377.83015)

<Figure size 1224x576 with 0 Axes>